In [ ]:
%load_ext autoreload
%autoreload
from curve_fitting.curve_fitting_class import curve_fitter
from matplotlib import pyplot as plt
from grad_shafranov_class import GS_Solution
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload
fit1 = curve_fitter(formfactor="Dshape", solverClass=GS_Solution)

In [ ]:
%load_ext autoreload
%autoreload
#plotting norm vector(gradient of psi)
Rgrid, Zgrid = fit1.flux_function.get_RZ_grid(20)
B_field_R, B_field_Z = fit1.flux_function.eval_del_psi(Rgrid, Zgrid)
plt.quiver(Rgrid, Zgrid, B_field_R, B_field_Z)
plt.show()

In [ ]:
%load_ext autoreload
%autoreload
# plotting psi contourmap
r_fs = 1.3
psi_fs = fit1.flux_function.eval_psi(r_fs, 0)
Rgrid, Zgrid = fit1.flux_function.get_RZ_grid(150)
fig, ax = plt.subplots(1, 1)
cp = ax.contourf(Rgrid, Zgrid, fit1.flux_function.eval_psi(Rgrid, Zgrid), levels=40, cmap='jet', alpha=0.9)
bounds = fit1.find_bounding_box(r_fs=r_fs, z_fs = 0)
r0 = fit1.flux_function.mid_point_finder(initial_R0=(bounds[0]+bounds[1])/2, initial_Z0=(bounds[2]+bounds[3])/2)
ax.plot(r0[0], r0[1], 'ro')

cp2 = ax.contour(Rgrid, Zgrid, fit1.flux_function.eval_psi(Rgrid, Zgrid), [psi_fs], colors='k', linewidths=2.,
                 linestyles='solid')




fig.colorbar(cp)  # Add a colorbar to a plot
ax.set_title('Psi functions evaluated(R,Z)')
ax.set_xlabel('R')
ax.set_ylabel('Z')
plt.show()

In [ ]:
%load_ext autoreload
%autoreload
# root searching and the plotting results
m_max_end = 12
psi_fs, x_final,xmap = fit1.find_flux_surface(r_fs=r_fs, m_max_start=2, m_max_end=12)
#res=results[("m_max=%d")%(12)]
#x_final,xmap=res["x_final"],res["xmap"]
#try setting theta offset to 1/2(dx)
#dx = 1/len(Np_in)
R,Z = fit1.flux_function.get_RZ_grid(150)
psi = fit1.flux_function.eval_psi(R,Z)
rr,zz= fit1.get_curve(psi_fs=psi_fs,x_final_in=x_final,xmap_in=xmap,Np_in=(2*m_max_end+1),rzcorr_in=1,thet_offset=0)
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.contourf(R, Z, psi, 40, cmap='jet', alpha=0.4)
ax.contour(R, Z, psi, 40, cmap='jet')
ax.contour(R,Z, psi, [psi_fs], colors='k', linewidths=2., linestyles='solid')
plt.plot(rr, zz, 'bo')
print("psi fs: ", psi_fs)
plt.xlabel('R')
ax.axis("equal")
plt.show()

In [ ]:
%load_ext autoreload
%autoreload
#inverse fourier transofrming the points after line search
coefficients,xmap = fit1.inverse_fourier_tranform(m_max_end=m_max_end,rr=rr,zz=zz)
rr_curve,zz_curve=fit1.eval_curve(xmap=xmap, x=coefficients, N=1024)

#printing average psi error for linesearched points
del_psi_points = np.average(np.abs(fit1.flux_function.eval_psi(rr,zz) - psi_fs*np.ones(len(rr))))
print("del_psi_points: " , del_psi_points)

#average psi error for curve after inverse fourier
del_psi_curve = np.average(np.abs(fit1.flux_function.eval_psi(rr_curve,zz_curve) - psi_fs))
print("del_psi_curve: " , del_psi_curve)